In [1]:
import pickle

import numpy as np
from attrdict import AttrDict
from RL.ppo import *
from utils.utils import log
from envs.cpu.env import make_env
from envs.brax_custom.gpu_env import make_vec_env_brax
from models.actor_critic import Actor
from models.vectorized import VectorizedActor

from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image
from brax import envs
from jax import numpy as jnp

pybullet build time: Dec 23 2020 01:49:05


In [2]:
%pwd
import os
os.chdir('/home/sbatra/QDPPO')

In [3]:
cfg = {'env_name': 'walker2d', 'env_batch_size': None, 'normalize_obs': False, 'normalize_rewards': True,
       'num_dims': 2, 'envs_per_model': 1, 'seed': 1111, 'obs_shape': (17,), 'action_shape': (6,), 'num_envs': 1}
cfg = AttrDict(cfg)

# # seeding
# seed = cfg.seed
# # seeding
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
env = make_vec_env_brax(cfg)

obs_shape, action_shape = env.observation_space.shape, env.action_space.shape
print(obs_shape, action_shape)

(17,) (6,)


In [4]:
# def load_agent_from_archive():
#     archive_path = '/home/sbatra/QDPPO/logs/method1-maega-walker2d-debug/cma_maega/trial_0/checkpoints/cp_00001000/archive_00001000.pkl'
#     with open(archive_path, 'rb') as f:
#         archive_df = pickle.load(f)
#     # elites = archive_df.query("objective > 6000").sort_values("objective", ascending=False)
#     agent_params = archive_df.query('2435').to_numpy()[4:]
#     agent = Actor(cfg, obs_shape=obs_shape, action_shape=action_shape).deserialize(agent_params).to(device)
#     return agent

In [5]:
def enjoy_brax(agent=None):
    if agent is None:
        agent = Actor(cfg, obs_shape, action_shape).to(device)
        cp_path = "checkpoints/walker2d_brax_model_0_checkpoint_debug"
        model_state_dict = torch.load(cp_path)['model_state_dict']
        model_state_dict['actor_logstd'] = model_state_dict['actor_logstd'].reshape(1, -1)
        agent.load_state_dict(model_state_dict)

    if cfg.normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var


    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if cfg.normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)
            act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act)
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew

    i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
    display(i) 
    print(f'{total_reward=}')
    print(f' Rollout length: {len(rollout)}')
    measures /= len(rollout)
    print(f'Measures: {measures.cpu().numpy()}')


In [6]:
# agent = load_agent_from_archive()
enjoy_brax()

total_reward=tensor(1016.5139, device='cuda:0')
 Rollout length: 1001
Measures: [0.99700296 0.99700296]
